In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as matplotlib
print(matplotlib.__version__)

3.3.4


In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
FIGSIZE = (12,5)

In [4]:
FEATURES = ['Реальные располагаемые денежные доходы населения',
            #'Среднегодовая остаточная стоимость облагаемого имущества',
            'Внешнеторговый оборот',
            'Численность постоянного населения',
            'Инвестиции в основной капитал',
            'Ввод в эксплуатацию жилых домов',
            'Оборот розничной торговли',
            'Объем платных услуг населению',
            'Фонд заработной платы',
            'Численность безработных, зарегистрированных в службах занятости',
            'Уровень зарегистрированной безработицы (к численности экономически активного населения)']
features = []

In [5]:
AGE_BEGIN = 2010
AGE_END = 2020
years = [year for year in range(AGE_BEGIN,AGE_END+1)]

In [6]:
PATH = './data/Основные параметры прогнозов СЭР/'

sed = {}

for year in years:
    try:
        df = pd.read_excel(f'{PATH}Основные параметры прогноза на {year}-{year+2} годы.xlsx', header=1)
    except:
        df = pd.read_excel(f'{PATH}Основные параметры прогноза на {year}-{year+2} годы.xls', header=1)
    sed[year] = df

C:\Users\Sergio\anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [7]:
for year in years:
    sed[year].rename(columns={f'{sed[year].columns[0]}':'indicator'}, inplace=True)

In [8]:
sed[2010]['indicator'].str.replace(" ",'') == FEATURES[1].replace(" ",'')

0     False
1     False
2     False
3     False
4     False
      ...  
66    False
67    False
68    False
69    False
70    False
Name: indicator, Length: 71, dtype: bool

In [9]:
sed[2010][sed[2010]['indicator'].str.replace(" ",'') == FEATURES[1].replace(" ",'')]

,indicator,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
62,Внешнеторговый оборот,млн.долл.\nСША,1758.30,1951.50,1272,1235,1280,1270,1350,1305,1430


In [10]:
sed[2015][sed[2015]['indicator'].str.replace(" ",'') == FEATURES[1].replace(" ",'')]

,indicator,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
58,Внешнеторговый оборот,млн.долл.\nСША,1564.70,1322.80,1515,1447,1550,1475,1600,1513,1665


In [11]:
sed[2015]['indicator'].str.replace(" ",'').iloc[-2]

'Численностьбезработных,зарегистрированныхвслужбахзанятости'

In [12]:
FEATURES[0].replace(" ",'')

'Реальныерасполагаемыеденежныедоходынаселения'

In [13]:
FEATURES[1].replace(" ",'') in sed[2015]['indicator'].str.replace(" ",'').iloc[-1]

False

In [14]:
sed[2011]['indicator'].str.replace(" ",'').str.contains(FEATURES[1].replace(" ",''), regex=False).fillna(False)

0     False
1     False
2     False
3     False
4     False
      ...  
67    False
68    False
69    False
70    False
71    False
Name: indicator, Length: 72, dtype: bool

In [15]:
sed[2011]['indicator'].str.replace(" ",'').str.contains(FEATURES[1].replace(" ",''), regex=False).fillna(False).iloc[-1]

False

In [16]:
for feature in FEATURES:
    #print(feature)
    tmp = []
    for year in years:
        #print(year)
        try:
            tmp.append(sed[year][sed[year]['indicator'].str.replace(" ",'').str.contains(feature.replace(" ",''), regex=False).fillna(False)].iloc[:,-2].values[0])
        except:
            tmp.append(np.nan)
    #print(tmp)
    features.append(tmp)

In [17]:
#features

In [18]:
# test = []
# for year in years:
#     print(year)
#     test.append(sed[year][sed[year]['indicator'].str.replace(" ",'').str.contains(FEATURES[0].replace(" ",''), regex=False).fillna(False)].iloc[:,-2].values[0])
# test

In [19]:
dic = {"Год": years}
dic.update({f'{f}': feature for (f, feature) in zip(FEATURES, features)})
df = pd.DataFrame(dic)
df

,Год,Реальные располагаемые денежные доходы населения,Внешнеторговый оборот,Численность постоянного населения,Инвестиции в основной капитал,Ввод в эксплуатацию жилых домов,Оборот розничной торговли,Объем платных услуг населению,Фонд заработной платы,"Численность безработных, зарегистрированных в службах занятости",Уровень зарегистрированной безработицы (к численности экономически активного населения)
0,2010,100.00,1305.00,679.80,19600,150.00,73500,24500,68900,12.60,3.30
1,2011,101.00,1527.00,677.70,27450,145.00,71400,21800,66200,12.70,3.39
2,2012,102.00,1995.00,633.50,38610,240.00,89600,24500,72760,9.90,2.66
3,2013,103.00,2070.00,633.10,44420,299.10,100000,31700,85400,8.20,2.30
4,2014,102.00,1440.00,631.40,47400,225.00,111000,32600,96500,7.50,2.27
5,2015,NaN,1513.00,628.20,43890,238.00,116000,35400,90100,6.90,2.21
6,2016,NaN,1079.80,623.90,43750,252.00,112000,39700,91300,7.50,2.33
7,2017,NaN,865.00,622.30,44560,260.00,127200,37600,90600,7.80,2.41
8,2018,NaN,1067.70,619.00,47300,280.00,124800,36600,89700,6.88,2.13
9,2019,NaN,2015.00,611.30,53500,260.00,136700,40700,102000,5.90,1.93


In [20]:
df.to_csv('./data/features.csv', index=False, encoding='utf-8-sig')